In [1]:
import torch
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
from matplotlib import rc
matplotlib.rcParams['figure.dpi'] = 150

import matplotlib.pyplot as plt
from sentence_transformers import models, SentenceTransformer
from collections import defaultdict
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

from scipy.stats import binned_statistic

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

epoch = 3 ver
data_idx = [0,1,2,3,4]

In [2]:
data_indices = [0,1,2,3,4]

In [3]:
model_types = ['model_bert_ft', 'model_roberta-base_ft', 'model_bert', 'model_roberta-base']#, 'model_sbert_ft', 'model_sbert'
model_labels = ['Finetuned BERT', 'Finetuned SBERT', 'BERT', 'SBERT'] #, 'Finetuned SBERT roberta', 'SBERT roberta'

In [4]:
def load_model(model_type, data_idx):
    
    if model_type == 'model_bert':
        bert = models.Transformer('bert-base-uncased')
        pooler = models.Pooling(
            bert.get_word_embedding_dimension(), #768
            pooling_mode_mean_tokens=True #mean pooling
        )
        model = SentenceTransformer(modules=[bert, pooler])    
    
    if model_type == 'model_bert_ft':
        MODEL_PATH = '../model/finetuned-BERT_idx%d_epoch3'%(data_idx)
        model = SentenceTransformer(MODEL_PATH)
    
    if model_type == 'model_roberta-base':
        model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
    
    if model_type == 'model_roberta-base_ft':
        MODEL_PATH = '../model/roberta-base_idx%d_epoch3'%(data_idx)
        model = SentenceTransformer(MODEL_PATH)

    return model

## User embedding

Goal: find user embeddings for each model and save them in the /dataset/UserEmbeddings/

 * Run __get_user_embeddings.py__ to get user belief embeddings and user embedding

## Classification task - test set

In [24]:
for data_idx in range(5):

    df_train = pd.read_pickle('../dataset/04_K-fold_dataset/df_train_idx%d.p'%(data_idx))
    df_test  = pd.read_pickle('../dataset/04_K-fold_dataset/df_test_idx%d.p'%(data_idx))

                              
    #3.1 Find common users who are included in both train and test data
    trainset_users = df_train['username'].unique()
    testset_users = df_test['username'].unique()
    
    common_users = []
    for user in testset_users:
        if user in trainset_users:
            common_users.append(user)
    
    print(f"{len(common_users)} users are included in both dataset.")       
    
    #3.2 generate dataframe composed of common user
    df_train_common = df_train[df_train['username'].isin(common_users)] 
    df_test_common = df_test[df_test['username'].isin(common_users)] 
    
    print(len(df_train_common), len(df_test_common))
    
    # Add vote length information  
    user2votelen = dict(df_train_common['username'].value_counts())
    lengths = list(user2votelen.values())
    df_train_common['votelen'] = df_train_common['username'].map(user2votelen)
    df_test_common['votelen'] = df_test_common['username'].map(user2votelen)

    df_train_common.to_pickle('../dataset/DownstreamTask_Dataframe/df_train_common_dataidx%d.p'%(data_idx))
    df_test_common.to_pickle('../dataset/DownstreamTask_Dataframe/df_test_common_dataidx%d.p'%(data_idx))

10173 users are included in both dataset.
111724 33208


/tmp/ipykernel_1062089/3257292800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_common['votelen'] = df_train_common['username'].map(user2votelen)
/tmp/ipykernel_1062089/3257292800.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_common['votelen'] = df_test_common['username'].map(user2votelen)


10401 users are included in both dataset.
110860 35171


/tmp/ipykernel_1062089/3257292800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_common['votelen'] = df_train_common['username'].map(user2votelen)
/tmp/ipykernel_1062089/3257292800.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_common['votelen'] = df_test_common['username'].map(user2votelen)


9706 users are included in both dataset.
107163 36060


/tmp/ipykernel_1062089/3257292800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_common['votelen'] = df_train_common['username'].map(user2votelen)
/tmp/ipykernel_1062089/3257292800.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_common['votelen'] = df_test_common['username'].map(user2votelen)


9675 users are included in both dataset.
113625 27733


/tmp/ipykernel_1062089/3257292800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_common['votelen'] = df_train_common['username'].map(user2votelen)
/tmp/ipykernel_1062089/3257292800.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_common['votelen'] = df_test_common['username'].map(user2votelen)


9744 users are included in both dataset.
110227 33007


/tmp/ipykernel_1062089/3257292800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_common['votelen'] = df_train_common['username'].map(user2votelen)
/tmp/ipykernel_1062089/3257292800.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_common['votelen'] = df_test_common['username'].map(user2votelen)


## Statistics

In [17]:
for data_idx in range(5):

    dat_train = pd.read_pickle('../dataset/DownstreamTask_Dataframe/df_train_common_dataidx%d.p'%(data_idx))
    dat_test  = pd.read_pickle('../dataset/DownstreamTask_Dataframe/df_test_common_dataidx%d.p'%(data_idx))
                              
    #3.1 Find common users who are included in both train and test data
    trainset_users = dat_train['username'].unique()
    testset_users = dat_test['username'].unique()
    
    print("data_idx: ", data_idx+1)
    print("# debates: " , len(dat_train['debate_title'].unique()), len(dat_test['debate_title'].unique()))
    print("# Users: " , len(trainset_users), len(testset_users))
    print("# Participation: " , len(dat_train), len(dat_test))
    print("# Participation per user: %.2f %.2f"%(len(dat_train)/len(trainset_users), len(dat_test)/len(testset_users)))
    print()

data_idx:  1
# debates:  42255 11581
# Users:  10173 10173
# Participation:  111724 33208
# Participation per user: 10.98 3.26

data_idx:  2
# debates:  42464 11571
# Users:  10401 10401
# Participation:  110860 35171
# Participation per user: 10.66 3.38

data_idx:  3
# debates:  40936 11720
# Users:  9706 9706
# Participation:  107163 36060
# Participation per user: 11.04 3.72

data_idx:  4
# debates:  42409 11327
# Users:  9675 9675
# Participation:  113625 27733
# Participation per user: 11.74 2.87

data_idx:  5
# debates:  41591 11682
# Users:  9744 9744
# Participation:  110227 33007
# Participation per user: 11.31 3.39



In [25]:
def get_rephrase(s, pro=True):
    pro_phrase = 'I agree with the following: '
    con_phrase = 'I disagree with the following: '

    if pro:
        return pro_phrase + s
    else:
        return con_phrase + s

In [8]:
"""for data_idx in range(5):
    for model_type in model_types:
        df_train = pd.read_pickle('../dataset/04_K-fold_dataset/df_train_idx%d.p'%(data_idx))
        df_test  = pd.read_pickle('../dataset/04_K-fold_dataset/df_test_idx%d.p'%(data_idx))

        with open('../dataset/UserEmbeddings/user2embeddingvector_data(%d)_model(%s).p'%(data_idx, model_type), 'rb' ) as f :   
            user2embedding = pickle.load(f)
    
    #print(len(df_train['username'].unique()), len(user2embedding) )"""

"for data_idx in range(5):\n    for model_type in model_types:\n        df_train = pd.read_pickle('../dataset/04_K-fold_dataset_nodup/df_train_idx%d.p'%(data_idx))\n        df_test  = pd.read_pickle('../dataset/04_K-fold_dataset_nodup/df_test_idx%d.p'%(data_idx))\n\n        with open('../dataset/UserEmbeddings/user2embeddingvector_data(%d)_model(%s).p'%(data_idx, model_type), 'rb' ) as f :   \n            user2embedding = pickle.load(f)\n    \n    #print(len(df_train['username'].unique()), len(user2embedding) )"

In [21]:
"""
user2avg_emb_train = {} #import user embeddings for each model for each data
for model_type in model_types:
    user2avg_emb_train[model_type] = {}
    for data_idx in data_indices:
        with open('../dataset/UserEmbeddings/user2embeddingvector_data(%d)_model(%s).p'%(data_idx, model_type), 'rb' ) as f :   
            user2avg_emb_train[model_type][data_idx] = pickle.load(f)
"""            

"\nuser2avg_emb_train = {} #import user embeddings for each model for each data\nfor model_type in model_types:\n    user2avg_emb_train[model_type] = {}\n    for data_idx in data_indices:\n        with open('../dataset/UserEmbeddings/user2embeddingvector_data(%d)_model(%s).p'%(data_idx, model_type), 'rb' ) as f :   \n            user2avg_emb_train[model_type][data_idx] = pickle.load(f)\n"

### Prediction task 

In [26]:
def cosine_accuracy(x_pro_vec, x_con_vec, user_vec):
    #cosine distance
    y_pro = []
    y_con = []
    for i in range(len(x_pro_vec)):
        a = cosine_similarity([x_pro_vec[i]], [user_vec[i]])[0][0]
        b = cosine_similarity([x_con_vec[i]], [user_vec[i]])[0][0]
        y_pro.append(a)
        y_con.append(b)
    y_pro = np.array(y_pro)
    y_con = np.array(y_con)
    y_hat = y_pro > y_con    

    acc   = sum(y == y_hat)/len(y)*100
    print("Acc(COS): %.2f%%"%(acc))
    return y_hat, acc

def euclidean_accuracy(x_pro_vec, x_con_vec, user_vec):
    #Euclidian distance
    y_pro_euc = []
    y_con_euc = []
    d_min,d_max,d_avg,d_rel = [],[],[],[]
    
    for i in range(len(x_pro_vec)):
        a = np.linalg.norm(x_pro_vec[i] - user_vec[i])
        b = np.linalg.norm(x_con_vec[i] - user_vec[i])
        y_pro_euc.append(a)
        y_con_euc.append(b)
        
        d_min.append(np.min([a,b]))
        d_max.append(np.max([a,b]))
        d_avg.append(np.mean([a,b]))
        d_rel.append(np.abs(a-b)/np.min([a,b]))
    
    y_pro_euc = np.array(y_pro_euc)
    y_con_euc = np.array(y_con_euc)
    y_hat_euc = y_pro_euc < y_con_euc    
    
    acc   = sum(y == y_hat_euc)/len(y)*100
    print("Acc(EUC): %.2f%%"%(acc))
    return y_hat_euc, acc, d_min,d_max,d_avg,d_rel

In [30]:
#for model_type in model_types:    
for model_type in model_types:    
    for data_idx in range(0,5):

        print(model_type, "data_idx",data_idx)
        
        #load data 
        df_sample = pd.read_pickle('../dataset/DownstreamTask_Dataframe/df_test_common_dataidx%d.p'%(data_idx))
        
        x = df_sample['debate_title']
        y = df_sample['position'] == 'Pro' #pro=1 con=0
        u = list(df_sample['username'])
        
        #generate two types of phrases for test set debate title 
        x_pro = [get_rephrase(e, pro=1) for e in x]
        x_con = [get_rephrase(e, pro=0) for e in x]
        
        #load model
        model = load_model(model_type, data_idx)
        device = "cuda:6" if torch.cuda.is_available() else "cpu"
        model.to(device)
        
        #generate pro/con vectors
        x_pro_vec = model.encode(x_pro)
        x_con_vec = model.encode(x_con)
    
        #get user embedding
        with open('../dataset/UserEmbeddings/user2embeddingvector_data(%d)_model(%s).p'%(data_idx, model_type), 'rb' ) as f :   
            user2embedding = pickle.load(f)
        
        user_vec  = [] 
        for e in u:
            user_vec.append(user2embedding[e])

        
        #Evaluation
        y_hat, acc, d_min, d_max, d_avg, d_rel = euclidean_accuracy(x_pro_vec, x_con_vec, user_vec)
     
        df_sample['predicted_outcome'] = y_hat        
        df_sample['true_outcome'] = df_sample['position']=='Pro'
        df_sample['d_min']= d_min
        df_sample['d_max']= d_max
        df_sample['d_avg']= d_avg
        df_sample['d_rel']= d_rel        

        df_sample.to_pickle('../dataset/DownstreamTask_Dataframe/downstream_results_data(%d)_model(%s).p'%(data_idx, model_type))
            


model_bert_ft data_idx 0
Acc(EUC): 56.83%
model_bert_ft data_idx 1
Acc(EUC): 57.20%
model_bert_ft data_idx 2
Acc(EUC): 56.21%
model_bert_ft data_idx 3
Acc(EUC): 55.12%
model_bert_ft data_idx 4
Acc(EUC): 56.27%
model_roberta-base_ft data_idx 0
Acc(EUC): 58.77%
model_roberta-base_ft data_idx 1
Acc(EUC): 59.29%
model_roberta-base_ft data_idx 2
Acc(EUC): 57.14%
model_roberta-base_ft data_idx 3
Acc(EUC): 56.18%
model_roberta-base_ft data_idx 4
Acc(EUC): 56.91%
model_bert data_idx 0
Acc(EUC): 54.68%
model_bert data_idx 1
Acc(EUC): 53.60%
model_bert data_idx 2
Acc(EUC): 54.28%
model_bert data_idx 3
Acc(EUC): 53.34%
model_bert data_idx 4
Acc(EUC): 53.53%
model_roberta-base data_idx 0
Acc(EUC): 56.71%
model_roberta-base data_idx 1
Acc(EUC): 55.99%
model_roberta-base data_idx 2
Acc(EUC): 56.39%
model_roberta-base data_idx 3
Acc(EUC): 55.25%
model_roberta-base data_idx 4
Acc(EUC): 56.35%


### baseline prediction - majority and random guess

In [3]:
from collections import Counter

def find_majority(x):
    cnter = Counter(x)
    return cnter.most_common()[0][0]
    

In [7]:
for data_idx in range(0,5):

    print("data_idx", data_idx)
    
    #load data 
    df_sample = pd.read_pickle('../dataset/DownstreamTask_Dataframe/df_test_common_dataidx%d.p'%(data_idx))

    #train data record
    df_before = pd.read_pickle('../dataset/DownstreamTask_Dataframe/df_train_common_dataidx%d.p'%(data_idx))

    user2majority = {}
    for user in df_sample['username'].unique():
        prev_positions = df_before[df_before['username']==user]['position']
        user2majority[user] = find_majority(prev_positions)
    
    
    prev_positions = df_before['position']
    group_majority_position = find_majority(prev_positions)
    
    user2majority_grouplevel = {}
    for user in df_sample['username'].unique():
        user2majority_grouplevel[user] = group_majority_position

    #predict user belief
    users = list(df_sample['username'])
    y_pred = []
    y_pred_group = []
    
    for u in users: 
        y_pred.append(user2majority[u]=='Pro')
        y_pred_group.append(user2majority_grouplevel[u] == 'Pro')

    for model_type in model_types:
        df_ = pd.read_pickle('../dataset/DownstreamTask_Dataframe/downstream_results_data(%d)_model(%s).p'%(data_idx, model_type))
        df_['majority_pred'] = y_pred
        df_['majority_pred_group'] = y_pred_group
        df_['random_pred'] = np.random.randint(0,2, size=len(users))
        df_.to_pickle('../dataset/DownstreamTask_Dataframe/downstream_results_data(%d)_model(%s).p'%(data_idx, model_type))
    
    

data_idx 0
data_idx 1
data_idx 2
data_idx 3
data_idx 4


In [8]:
df_.head()

,debate_key,debate_title,username,debate_date,position,is_belief,belief_statement,votelen,predicted_outcome,true_outcome,d_min,d_max,d_avg,d_rel,majority_pred,random_pred,majority_pred_group
63730,The-Democratic-Peoples-Republic-of-Korea-is-no...,The Democratic People's Republic of Korea is n...,SteveMaxwell,2015-10-25,Pro,1.0,I agree with the following: The Democratic Peo...,1,True,True,24.716259,24.834814,24.775536,0.004797,True,0,False
63731,The-democrats-are-more-racist-than-the-republi...,The democrats are more racist than the republi...,Flipbook,2014-11-24,Pro,1.0,I agree with the following: The democrats are ...,8,False,True,16.578342,18.275551,17.426947,0.102375,False,0,False
63732,The-Democrats-are-not-protecting-the-US-Consti...,The Democrats are not protecting the US Consti...,Flipbook,2013-05-14,Pro,1.0,I agree with the following: The Democrats are ...,8,False,True,16.456379,17.252188,16.854282,0.048359,False,0,False
63733,The-Democrats-scare-me./1/,The Democrats scare me.,wingnut2280,2008-01-22,Pro,1.0,I agree with the following: The Democrats scar...,42,False,True,13.721004,15.272534,14.496769,0.113077,False,0,False
63734,The-democrats-show-their-true-colors-stalinist...,The democrats show their true colors - stalini...,Solarman1969,2008-02-15,Pro,1.0,I agree with the following: The democrats show...,55,True,True,15.502457,16.059826,15.781141,0.035954,True,1,False
